# 10 数据聚合与分组操作
1 使用一个或多个键将pandas对象拆分
2 计算组内汇总统计信息，如计数，平均值或标准差或用户定义的函数
3 应用组内变换或其他操作，如标准化、线性回归、排位或子集选择
4 计算数据透视表和交叉表
5 执行分位数分析和其他统计组分析

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1': list('aabba'),
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [3]:
# 根据key1计算data1的平均值
df['data1'].groupby(df['key1']).mean()

key1
a   -0.427583
b    0.728017
Name: data1, dtype: float64

In [4]:
# 也可以使用多列进行groupby，传入一个列表
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one    -0.919364
      two     0.555978
b     one     1.354937
      two     0.101097
Name: data1, dtype: float64

In [5]:
# 改变顺序不影响最终结果
# groupby和mean操作之后发生了什么？
# 首先根据分组键进行了聚合，并产生一个新的Series，
# 这个Series使用key1，key2组合后的唯一值作为索引，然后计算data1对应索引上的均值
df.groupby([df['key1'], df['key2']])['data1'].mean()

key1  key2
a     one    -0.919364
      two     0.555978
b     one     1.354937
      two     0.101097
Name: data1, dtype: float64

In [6]:
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [8]:
# 分组键可以是正确长度的任意数组
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2006, 2005, 2006, 2005])
df.groupby([states, years])['data1'].mean()

California  2005    1.354937
            2006    0.555978
Ohio        2005   -0.919364
            2006    0.101097
Name: data1, dtype: float64

In [9]:
# 传递列名也可以作为分组键
# 可以看到结果中并没有key2，因为key2并不是数值数据
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.427583,-0.464749
b,0.728017,-0.467009


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.919364 -1.152596
     two   0.555978  0.910947
b    one   1.354937 -0.938447
     two   0.101097  0.004430

In [13]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 遍历各分组

In [14]:
# groupby对象支持迭代，会生成一个包含组名和数据块的2维元组序列
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.844591 -1.393349
1    a  two  0.555978  0.910947
4    a  one -0.994137 -0.911844
b
  key1 key2     data1     data2
2    b  one  1.354937 -0.938447
3    b  two  0.101097  0.004430


In [15]:
# 在多个分组键的情况下，元组中的第一个元素是键值的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.844591 -1.393349
4    a  one -0.994137 -0.911844
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.555978  0.910947
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.354937 -0.938447
('b', 'two')
  key1 key2     data1    data2
3    b  two  0.101097  0.00443


In [16]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.844591 -1.393349
 1    a  two  0.555978  0.910947
 4    a  one -0.994137 -0.911844,
 'b':   key1 key2     data1     data2
 2    b  one  1.354937 -0.938447
 3    b  two  0.101097  0.004430}

### 10.1.2 选择一列或所有列的子集

In [18]:
# 只有data1
df.groupby('key1')['data1'].sum()

key1
a   -1.282750
b    1.456034
Name: data1, dtype: float64

In [21]:
df['data1'].groupby(df['key1']).sum()

key1
a   -1.282750
b    1.456034
Name: data1, dtype: float64

In [22]:
# 选择一部分数据列，只需要指定就好
df.groupby(['key1', 'key2'])['data2'].sum()

key1  key2
a     one    -2.305193
      two     0.910947
b     one    -0.938447
      two     0.004430
Name: data2, dtype: float64

In [23]:
# 1 如果只有单个列名作为标量传递，则为分组的Series
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [24]:
s_grouped.mean()

key1  key2
a     one    -1.152596
      two     0.910947
b     one    -0.938447
      two     0.004430
Name: data2, dtype: float64

In [26]:
# 2 如果传递的是列表或数组，则此索引操作返回的对象是分组的DataFrame
# 传入列表，返回DataFrame
s_grouped = df.groupby('key1')[['data2']]
s_grouped

### 10.1.3 使用字典和Series分组

In [27]:
# 分组信息可能会以非数组形式存在
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=list('abcde'),
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,-0.405816,-0.669264,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [29]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,NaN,NaN,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [30]:
# 各列的分组对应关系，并且想把各列按组累加
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [31]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.730774,-2.881193
Steve,2.650345,2.166053
Wes,0.104635,1.175141
Jim,-0.818363,-1.435931
Travis,0.126143,-1.671693


In [32]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

### 10.1.4 使用函数分组

In [33]:
# 按照名字的长度分组
people.groupby(len).sum()

,a,b,c,d,e
3,0.709638,-0.886556,1.641493,-1.624447,-2.965064
5,0.736930,0.450232,1.831150,0.819195,0.978891
6,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [34]:
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,NaN,NaN,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [35]:
# 将函数与数组、字典或Series进行混合并不困难，所有的对象那个都会在内部转换为数组
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).sum()

a         b         c         d         e
3 one  1.515352 -2.508598  0.709008  0.126401 -0.712806
  two -0.805714  1.622042  0.932485 -1.750847 -2.252258
5 one  0.736930  0.450232  1.831150  0.819195  0.978891
6 two -0.590660 -0.622607  1.144758 -1.018615 -0.458426

### 10.1.5 根据索引层级分组

In [36]:
# 分层索引的数据有一个非常方便的地方，就是能够在周索引的某个层级上进行聚合
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.069803 -0.845073  2.057548 -1.672885 -0.760306
1      1.250289  1.074574  1.148241  1.187818  0.576493
2      0.350411 -0.100084  0.406645  0.983204 -0.178269
3     -0.485553 -1.253105  0.632496 -0.850636 -0.401723

In [37]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 数据聚合

In [38]:
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [41]:
df.groupby('key1')['data1'].quantile(0.5)

key1
a   -0.844591
b    0.728017
Name: data1, dtype: float64

In [43]:
# 要使用子集的聚合函数，需要将函数传递给aggregate或agg方法
def peak_to_peak(arr):
    return arr.max() - arr.min()


df.groupby('key1')['data1'].agg(peak_to_peak)

key1
a    1.550115
b    1.253841
Name: data1, dtype: float64

In [45]:
# 列出groupby默认的所有聚合结果
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.427583  0.855065 -0.994137 -0.919364 -0.844591 -0.144306   
b      2.0  0.728017  0.886599  0.101097  0.414557  0.728017  1.041477   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.555978   3.0 -0.464749  1.215469 -1.393349 -1.152596 -0.911844   
b     1.354937   2.0 -0.467009  0.666714 -0.938447 -0.702728 -0.467009   

                          
           75%       max  
key1                      
a    -0.000449  0.910947  
b    -0.231290  0.004430

### 10.2.1 逐列及多函数应用

In [46]:
tips = pd.read_csv('./examples/tips.csv')
tips['tip_pct'] = tips['tip'] / (tips['total_bill'])
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [47]:
grouped = tips.groupby(['smoker', 'day'])
grouped.agg('mean')

total_bill       tip      size   tip_pct
smoker day                                           
No     Fri    18.420000  2.812500  2.250000  0.151650
       Sat    19.661778  3.102889  2.555556  0.158048
       Sun    20.506667  3.167895  2.929825  0.160113
       Thur   17.113111  2.673778  2.488889  0.160298
Yes    Fri    16.813333  2.714000  2.066667  0.174783
       Sat    21.276667  2.875476  2.476190  0.147906
       Sun    24.120000  3.516842  2.578947  0.187250
       Thur   19.190588  3.030000  2.352941  0.163863

In [49]:
# 你可以将函数名以字符串形式传递
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

smoker  day 
No      Fri     0.151650
        Sat     0.158048
        Sun     0.160113
        Thur    0.160298
Yes     Fri     0.174783
        Sat     0.147906
        Sun     0.187250
        Thur    0.163863
Name: tip_pct, dtype: float64

In [50]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
smoker day                                   
No     Fri   0.151650  0.028123      0.067349
       Sat   0.158048  0.039767      0.235193
       Sun   0.160113  0.042347      0.193226
       Thur  0.160298  0.038774      0.193350
Yes    Fri   0.174783  0.051293      0.159925
       Sat   0.147906  0.061375      0.290095
       Sun   0.187250  0.154134      0.644685
       Thur  0.163863  0.039389      0.151240

In [52]:
# 传递的是(name, function)元组的列表，每个元组的第一个元素将作为DataFrame的列名
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
smoker day                     
No     Fri   0.151650  0.028123
       Sat   0.158048  0.039767
       Sun   0.160113  0.042347
       Thur  0.160298  0.038774
Yes    Fri   0.174783  0.051293
       Sat   0.147906  0.061375
       Sun   0.187250  0.154134
       Thur  0.163863  0.039389

In [54]:
# 在DataFrame，可以指定应用的所有列上的函数列表或每一列上应用的不同函数
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
smoker day                                                          
No     Fri        4  0.151650  0.187735          4  18.420000  22.75
       Sat       45  0.158048  0.291990         45  19.661778  48.33
       Sun       57  0.160113  0.252672         57  20.506667  48.17
       Thur      45  0.160298  0.266312         45  17.113111  41.19
Yes    Fri       15  0.174783  0.263480         15  16.813333  40.17
       Sat       42  0.147906  0.325733         42  21.276667  50.81
       Sun       19  0.187250  0.710345         19  24.120000  45.35
       Thur      17  0.163863  0.241255         17  19.190588  43.11

In [55]:
result['tip_pct']

count      mean       max
smoker day                            
No     Fri       4  0.151650  0.187735
       Sat      45  0.158048  0.291990
       Sun      57  0.160113  0.252672
       Thur     45  0.160298  0.266312
Yes    Fri      15  0.174783  0.263480
       Sat      42  0.147906  0.325733
       Sun      19  0.187250  0.710345
       Thur     17  0.163863  0.241255

In [56]:
# 也可以传递(name, function)
ftuples = [('A', 'mean'), ('B', 'max')]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill       
                    A         B          A      B
smoker day                                       
No     Fri   0.151650  0.187735  18.420000  22.75
       Sat   0.158048  0.291990  19.661778  48.33
       Sun   0.160113  0.252672  20.506667  48.17
       Thur  0.160298  0.266312  17.113111  41.19
Yes    Fri   0.174783  0.263480  16.813333  40.17
       Sat   0.147906  0.325733  21.276667  50.81
       Sun   0.187250  0.710345  24.120000  45.35
       Thur  0.163863  0.241255  19.190588  43.11

In [57]:
# 在不同列上应用不同的函数
grouped.agg({'tip': np.max, 'size': sum})

tip  size
smoker day              
No     Fri    3.50     9
       Sat    9.00   115
       Sun    6.00   167
       Thur   6.70   112
Yes    Fri    4.73    31
       Sat   10.00   104
       Sun    6.50    49
       Thur   5.00    40

In [59]:
grouped.agg({'tip': ['min', 'max', 'mean', 'std'], 'size': ['sum', 'count']})

tip                            size      
              min    max      mean       std  sum count
smoker day                                             
No     Fri   1.50   3.50  2.812500  0.898494    9     4
       Sat   1.00   9.00  3.102889  1.642088  115    45
       Sun   1.01   6.00  3.167895  1.224785  167    57
       Thur  1.25   6.70  2.673778  1.282964  112    45
Yes    Fri   1.00   4.73  2.714000  1.077668   31    15
       Sat   1.00  10.00  2.875476  1.630580  104    42
       Sun   1.50   6.50  3.516842  1.261151   49    19
       Thur  2.00   5.00  3.030000  1.113491   40    17

In [61]:
# 返回不含索引的聚合数据
no_index = tips.groupby(['day', 'smoker'], as_index=False).mean()
no_index

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [62]:
type(no_index)

pandas.core.frame.DataFrame


## 10.3 应用：通用拆分(split)-应用(apply)-联合(combine)

In [63]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [64]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]


top(tips)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [65]:
# 按照smoker进行分组，之后调用apply
# top函数在DataFrame的每一行分组上被调用，之后使用pandas.concat将函数结果粘贴在一起
# 并使用分组名作为各组的标签，因此结果包含分层索引，该层索引的内部层级包含原DataFrame的索引值
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [66]:
# 如果要传入其他参数或关键字，可以放在函数后进行传递
tips.groupby('smoker').apply(top, n=1, column='total_bill')

,,total_bill,tip,smoker,day,time,size,tip_pct
smoker,,,,,,,,
No,212,48.33,9.0,No,Sat,Dinner,4,0.186220
Yes,170,50.81,10.0,Yes,Sat,Dinner,3,0.196812


In [68]:
# 除了基础的使用机制，需要一些创造力才能充分地使用apply。传递函数的内部发生的事情取决于子集，函数只返回一个pandas对应或一个标量值。
# 之前在groupby对象上调用了describe方法
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [70]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [72]:
# 在Groupby对象的内部，当你调用箱describe这样的方法时，实际上是以下代码的简写：
f = lambda x: x.describe()
grouped.apply(f)[:5]

total_bill       tip  size   tip_pct
smoker day                                            
No     Fri count    4.000000  4.000000  4.00  4.000000
           mean    18.420000  2.812500  2.25  0.151650
           std      5.059282  0.898494  0.50  0.028123
           min     12.460000  1.500000  2.00  0.120385
           25%     15.100000  2.625000  2.00  0.137239

### 10.3.1 压缩分组键

In [73]:
# 在之前例子中，可以看到得到的对象具有分组键所形成的分层索引以及每个原始对象的索引
# 可以通过传递group_keys=False来禁用这个功能
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 分位数与桶分析

In [77]:
# cut和qcut
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:5]

0     (-3.001, -1.504]
1     (-0.0135, 1.477]
2     (-0.0135, 1.477]
3    (-1.504, -0.0135]
4       (1.477, 2.968]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.001, -1.504] < (-1.504, -0.0135] < (-0.0135, 1.477] < (1.477, 2.968]]

In [78]:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

In [81]:
# 使用分位数区间来分组DataFrame
grouped = frame.data2.groupby(quartiles)
# 将分组后的DataFrame应用get_stats函数
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.001, -1.504]",-2.511018,2.888787,61.0,0.027127
"(-1.504, -0.0135]",-2.991909,3.035505,422.0,0.032793
"(-0.0135, 1.477]",-3.019117,2.524191,451.0,-0.015068
"(1.477, 2.968]",-1.859108,1.977192,66.0,-0.260528


In [86]:
# 上面的分桶为"等长桶"，为了根据样本分位桶计算出"等大小的桶"，需要使用qcut
grouping = pd.qcut(frame.data1, 10, labels=False)  # 传入10表示10分位
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.511018,2.888787,100.0,0.034311
1,-2.915698,3.035505,100.0,-0.023259
2,-2.401430,2.158680,100.0,-0.002027
3,-2.991909,2.741706,100.0,0.122747
4,-2.043449,2.710128,100.0,0.017067
5,-3.019117,2.327372,100.0,-0.100739
6,-2.937367,2.097503,100.0,0.119491
7,-2.899614,2.524191,100.0,0.045425
8,-2.648311,2.478626,100.0,-0.060017


### 10.3.3 示例：使用指定分组值填充缺失值

In [88]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.979274
2         NaN
3    0.122541
4         NaN
5   -0.901896
dtype: float64

In [89]:
# 使用均值填充NA
s.fillna(s.mean())

0   -0.919543
1   -1.979274
2   -0.919543
3    0.122541
4   -0.919543
5   -0.901896
dtype: float64

In [90]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [91]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.797169
New York      1.104516
Vermont       0.354852
Florida       0.168654
Oregon       -0.041550
Nevada        1.195868
California   -0.363097
Idaho         0.085700
dtype: float64

In [92]:
# 将一些值置为NA
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.797169
New York      1.104516
Vermont            NaN
Florida       0.168654
Oregon       -0.041550
Nevada             NaN
California   -0.363097
Idaho              NaN
dtype: float64

In [93]:
data.groupby(group_key).mean()

East    0.158667
West   -0.202324
dtype: float64

In [94]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda x: x.fillna(fill_values[x.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.797169
New York      1.104516
Vermont       0.500000
Florida       0.168654
Oregon       -0.041550
Nevada       -1.000000
California   -0.363097
Idaho        -1.000000
dtype: float64

### 10.3.4 示例：随机采样与排列

In [100]:
suits = list('HSCD')
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + list('JQK')
cards = []
for suit in list('HSCD'):
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [106]:
# 从这副牌中拿出五张牌
def draw(deck, n=5):
    return deck.sample(n)


draw(deck)

10H    10
9S      9
JD     10
AC      1
9H      9
dtype: int64

In [113]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   AC     1
D  9D     9
   QD    10
H  QH    10
   8H     8
S  JS    10
   7S     7
dtype: int64

In [115]:
deck.groupby(get_suit).describe()

,count,mean,std,min,25%,50%,75%,max
C,13.0,6.538462,3.281651,1.0,4.0,7.0,10.0,10.0
D,13.0,6.538462,3.281651,1.0,4.0,7.0,10.0,10.0
H,13.0,6.538462,3.281651,1.0,4.0,7.0,10.0,10.0
S,13.0,6.538462,3.281651,1.0,4.0,7.0,10.0,10.0


### 10.3.5 示例：分组加权平均和相关性

In [116]:
df = pd.DataFrame({'category': list('aaaabbbb'),
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.472005,0.320131
1,a,-0.296827,0.144299
2,a,-0.335415,0.081769
3,a,-0.212100,0.605111
4,b,0.990875,0.218523
5,b,0.070163,0.608457
6,b,0.593190,0.938680
7,b,-0.383460,0.953651


In [117]:
# 通过category进行分组加权平均
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.303746
b    0.165611
dtype: float64

In [121]:
# 股票的收盘价
close_px = pd.read_csv('./examples/stock_px_2.csv',
                       parse_dates=True,
                       index_col=0)
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [122]:
# 计算标普指数（SPX）每日收益的年度相关性（通过百分比幻化计算）。作为一种实现方式，首先创建一个每列与SPX列成对关联的函数
spx_corr = lambda x: x.corrwith(x['SPX'])

In [125]:
# 之后使用pct_change计算close-px百分比的变化
rets = close_px.pct_change().dropna()
rets[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974
2011-10-14,0.033225,0.003311,0.022784,0.017380


In [126]:
# 按年对百分比变化进行分组，可以使用单行函数从每个行标签中提取每个datetime标签的year属性
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [127]:
# 也可以计算内部相关性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 10.3.6 示例：逐组线性回归

In [128]:
import statsmodels.api as sm


def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [129]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
